In [25]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [26]:
my_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

dataset = torchvision.datasets.ImageFolder(root='shape_data', transform=my_transform)

In [27]:
dataset_loader=torch.utils.data.DataLoader(dataset, batch_size=64)

In [28]:
class MyConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=2)
        self.conv2=nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5)
        self.conv3=nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.conv4=nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.fc1=nn.Linear(32*18*18,3)
        self.relu=nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x=self.relu(self.conv1(x))
        x=self.relu(self.conv2(x))
        x=self.relu(self.conv3(x))
        x=self.relu(self.conv4(x))
        
        x=x.view(-1,18*18*32)
        x=self.softmax(self.fc1(x))
        return x
    


In [29]:
net2 = MyConv()
opt = optim.Adam(net2.parameters(), 0.00015)
loss_fn = nn.MSELoss()
losses = []
accuracies = [0.1]

for epoch in range(100):
    total_correct = 0
    total = len(dataset)
    epoch_loss = 0
    for images, labels in dataset_loader:
        hot_predictions = net2(images)
        predictions = hot_predictions.argmax(axis=1)
        nb_correct = sum(predictions == labels)
        total_correct += nb_correct
        
        hot_labels = F.one_hot(labels, 3).to(torch.float32)
        loss = loss_fn(hot_predictions, hot_labels)
        loss.backward()
        epoch_loss += loss.item()
        opt.step()
        opt.zero_grad()
        losses.append(epoch_loss)
    

    acc = total_correct / total
    accuracies.append(acc.item())
    print(epoch, acc)

plt.plot(losses)
plt.show()

0 tensor(0.2551)
1 tensor(0.3632)


KeyboardInterrupt: 

In [ ]:
##dummy_input = torch.ones(1,3,64,64)

In [ ]:
#torch.onnx.export(model, dummy_input, "adrian1.onnx", export.params = True, verbose=True)